In [20]:
import pandas as pd
import os


# Wczytanie danych

In [18]:
fp = 'input/254516_online.csv'
data = pd.read_csv(fp)
data.head()

,status pozyczki,kwota wnioskowana,kwota otrzymana,PESEL klienta,rodzaj źródła dochodu,miesieczny dochod netto,ktore rolowanie,data decyzji,data początkowa dochodu,data koncowa dochodu,planowany termin splaty,rzeczywisty termin spłaty
0,splacona_cz,800.0,800.0,490702xxx4x,13,3700.00,3,2011-08-31 16:26:31,2009-07-13,0000-00-00,2011-09-30,2011-09-30 00:00:00
1,splacona_cz,300.0,300.0,480124xxx2x,13,1100.00,0,2011-08-31 17:37:10,2003-01-01,0000-00-00,2011-09-30,2011-09-30 00:00:00
2,splacona_cz,400.0,400.0,460902xxx5x,16,1473.04,0,2011-09-01 09:15:50,1998-01-01,0000-00-00,2011-10-01,2011-09-30 00:00:00
3,splacona_cz,400.0,350.0,330729xxx3x,13,1519.00,0,2011-09-01 09:32:40,1990-08-16,0000-00-00,2011-10-01,2011-10-01 00:00:00
4,splacona_cz,200.0,200.0,741203xxx8x,12,599.03,2,2011-09-01 09:51:44,2010-03-01,2011-09-30,2011-10-01,2011-10-03 00:00:00


# Ćwiczenie 1

In [11]:
data['kwota kredytu'] = data.apply(lambda row: row['kwota wnioskowana'] if row['status pozyczki'] == 'odmowa' else row['kwota otrzymana'], axis=1)
data[['status pozyczki', 'kwota wnioskowana', 'kwota otrzymana', 'kwota kredytu']].head(10)

,status pozyczki,kwota wnioskowana,kwota otrzymana,kwota kredytu
0,splacona_cz,800.00,800.00,800.00
1,splacona_cz,300.00,300.00,300.00
2,splacona_cz,400.00,400.00,400.00
3,splacona_cz,400.00,350.00,350.00
4,splacona_cz,200.00,200.00,200.00
5,splacona_cz,200.00,200.00,200.00
6,splacona_cz,200.00,200.00,200.00
7,splacona_cz,218.32,218.32,218.32
8,splacona_cz,393.99,393.99,393.99
9,odmowa,1000.00,0.00,1000.00


# Ćwiczenie 2

In [12]:
def calculate_birth_date(pesel):
    year, month, day = int(pesel[:2]), int(pesel[2:4]), pesel[4:6]
    if month > 12:
        year += 2000
        month -= 20
    else:
        year += 1900
    month = f"{month:02d}"
    return f"{year}-{month}-{day}"


test_pesel = "012820xxx5x"
print(calculate_birth_date(test_pesel))

data['data urodzenia'] = data['PESEL klienta'].apply(calculate_birth_date)
data[['PESEL klienta', 'data urodzenia']].head()

2001-08-20


,PESEL klienta,data urodzenia
0,490702xxx4x,1949-07-02
1,480124xxx2x,1948-01-24
2,460902xxx5x,1946-09-02
3,330729xxx3x,1933-07-29
4,741203xxx8x,1974-12-03


# Ćwiczenie 3

In [13]:
data['data decyzji'] = pd.to_datetime(data['data decyzji'])
data['data urodzenia'] = pd.to_datetime(data['data urodzenia'])

data['wiek'] = data['data decyzji'].dt.year - data['data urodzenia'].dt.year
data[['data decyzji', 'data urodzenia', 'wiek']].head()


,data decyzji,data urodzenia,wiek
0,2011-08-31 16:26:31,1949-07-02,62
1,2011-08-31 17:37:10,1948-01-24,63
2,2011-09-01 09:15:50,1946-09-02,65
3,2011-09-01 09:32:40,1933-07-29,78
4,2011-09-01 09:51:44,1974-12-03,37


# Ćwiczenie 4

In [14]:
# Utworzenie kolumny 'płeć' na podstawie 10. cyfry numeru PESEL
data['płeć'] = data['PESEL klienta'].apply(lambda x: 'kobieta' if int(x[9]) % 2 == 0 else 'mężczyzna')

data[['PESEL klienta', 'płeć']].head()


,PESEL klienta,płeć
0,490702xxx4x,kobieta
1,480124xxx2x,kobieta
2,460902xxx5x,mężczyzna
3,330729xxx3x,mężczyzna
4,741203xxx8x,kobieta


# Ćwiczenie 5

In [17]:
# Poprawiam nazwę kolumny i ponawiam konwersję oraz obliczenia
data['data początkowa dochodu'] = pd.to_datetime(data['data początkowa dochodu'])

# Obliczanie różnicy w miesiącach
data['okres w jakim pobierał dochód'] = data.apply(lambda row:
                                                   (row['data decyzji'].year - row['data początkowa dochodu'].year) * 12 +
                                                   (row['data decyzji'].month - row['data początkowa dochodu'].month), axis=1)

# Sprawdzam wyniki
data[['data decyzji', 'data początkowa dochodu', 'okres w jakim pobierał dochód']].head()


,data decyzji,data początkowa dochodu,okres w jakim pobierał dochód
0,2011-08-31 16:26:31,2009-07-13,25
1,2011-08-31 17:37:10,2003-01-01,103
2,2011-09-01 09:15:50,1998-01-01,164
3,2011-09-01 09:32:40,1990-08-16,253
4,2011-09-01 09:51:44,2010-03-01,18


# Ćwiczenie 6

# Zapisanie danych

In [22]:
refactored_fp = 'output/254516_refactored.csv'

output_dir = os.path.dirname(refactored_fp)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

data.to_csv(refactored_fp, index=False)